In [ ]:
# !git clone https://github.com/fmilthaler/FinQuant.git
# !pip install FinQuant

In [ ]:
"""" FinQuant """
import pathlib
from finquant.portfolio import *
from finquant.moving_average import compute_ma, ema
import pandas as pd
import datetime

""" Remember to somehow account for price differences between currencies, namely
GBp to USD, and Yen """
names = ['GOOG', 'AMZN', 'OCDO.L', 'DIS', 'TSLA', 'WM', 'MMM', 'BTC-USD']
# London stocks need to be accounted for currency
pf = build_portfolio(names=names, data_api="yfinance", )

""" This will give you rfr, ER, vol, Sharpe, skew and kurt of equal wight pf """
# pf.properties()




[*********************100%***********************]  8 of 8 completed


' This will give you rfr, ER, vol, Sharpe, skew and kurt of equal wight pf '

In [ ]:
""" Build ema graph of a stock with moving average windows"""
dis = pf.get_stock("DIS").data.copy(deep=True)
dis1 = dis.iloc[14000:]                             # Split data to more recent
spans = [10, 50, 100, 150, 200]

ma = compute_ma(dis1, ema, spans, plot=True)


In [ ]:
""" Basic optimisation by minimising vol and maximising Sharpe """
opt_w, opt_res = pf.mc_optimisation(num_trials=5000)      # Monte Carlo weights and results
print(opt_res)                                            # Plot stats for optimals

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # Make pandas return whole df
    print(opt_w)

plt.figure(figsize=(22,14))
pf.mc_plot_results()

In [ ]:
""" Show indiv. stocks """
plt.figure(figsize=(22,14))

pf.ef_plot_efrontier()
pf.ef_plot_optimal_portfolios()
pf.plot_stocks()

The OOP of FinQuant
The public classes of Stock (individual objects) and Portfolio (groups of stock objects)

These classes can calculate:


*   Cumulative returns on pf
*   Daily returns (% and log)
*   Expected annualised return
*   Volatility, Skew and Kurtosis
*   Sharpe Ratio
*   Covariance matrix
*   Daily returns

And can be optimised for:


*   Min volatility 
*   Max Sharpe
*   ***Min vol, expected return***
*   ***Max Sharpe, given vol***

Also, software can plot monte carlos, efficient frontiers, min vol given numerical optimisation or expected return, max sharpe give numerical optimisation or target volatility. 


In [ ]:
""" When reading in, portfolio data is just a dictionary of 
{0: {'Name': 'Ticker', 'Allocation': Number},
{1: {'Name': 'Ticker', 'Allocation': Number},
{n: {'Name': 'Ticker', 'Allocation': Number}
}
Alternatively parse in 2 csv files....
"""


df_data = pd.read_csv('/content/Pfolio.csv', index_col='Date', parse_dates=True)
df_data.drop(columns=['SP500'], inplace=True)     # Drop S&P as not ticker
pf_data = pd.read_csv('/content/Pfolio_weights.csv')

""" Create the portfolio object """
pf2 = build_portfolio(data=df_data, pf_allocation=pf_data)

In [ ]:
""" Analyse portfolio object """
print(pf2)
pf2.properties()

print(pf2.comp_cumulative_returns())
print(pf2.comp_daily_log_returns())

pf2.comp_daily_log_returns().plot().axhline(y=0, color='black')

In [ ]:
""" Single Stocks """
tsla = pf2.get_stock("TSLA")
tsla_prices = tsla.data
print(tsla.properties())

In [ ]:
print(pf2.data.loc[str(datetime.datetime(2018, 1, 1))])
print(pf2.data.loc[pf2.data.index > datetime.datetime(2020, 1, 1)])

In [ ]:
# Max Sharpe, target vol
pf2.ef_efficient_volatility(0.15, verbose=True)

----------------------------------------------------------------------
Optimised portfolio for Efficient Volatility

Time window/frequency: 252
Risk free rate: 0.005
Expected annual Return: 0.807
Annual Volatility: 0.619
Sharpe Ratio: 1.296

Optimal weights:
                      FB          NFLX  TSLA           JPM
Allocation  2.457699e-15  1.751073e-13   1.0  2.521474e-15
----------------------------------------------------------------------


,Allocation
FB,2.457699e-15
NFLX,1.751073e-13
TSLA,1.000000e+00
JPM,2.521474e-15
